In [22]:

import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
from google.colab import drive, files
import os

EDGES = {
    (0, 1): 'm', (0, 2): 'c',
    (1, 3): 'm', (2, 4): 'c',
    (0, 5): 'm', (0, 6): 'c',
    (5, 7): 'm', (7, 9): 'm',
    (6, 8): 'c', (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm', (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm', (13, 15): 'm',
    (12, 14): 'c', (14, 16): 'c'
}


def draw_keypoints(frame, keypoints, confidence_threshold=0.3):
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    for ky, kx, kp_conf in shaped:
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 5, (0, 255, 0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold=0.3):
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    for (p1, p2) in edges:
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        if c1 > confidence_threshold and c2 > confidence_threshold:
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold=0.3):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

print("Loading MoveNet model...")
model = hub.load("https://tfhub.dev/google/movenet/multipose/lightning/1")
movenet = model.signatures["serving_default"]
print("Model loaded successfully!")


drive.mount('/content/drive')


video_path = "/content/drive/MyDrive/novak.mp4"

if not os.path.exists(video_path):
    raise FileNotFoundError(f"Input video not found at: {video_path}")

print("Input video found!")


cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError("Cannot open input video")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = 25  # Fixed FPS for Colab

print(f"Video properties: {width}x{height}, {fps} FPS")

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("movenet_output.avi", fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError("VideoWriter failed to open")

print("VideoWriter ready!")


frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video reached")
        break

    frame_count += 1

    img = tf.image.resize_with_pad(tf.expand_dims(frame, axis=0), 384, 640)
    input_img = tf.cast(img, dtype=tf.int32)

    results = movenet(input_img)
    keypoints_with_scores = results["output_0"].numpy()
    keypoints_with_scores = keypoints_with_scores[:, :, :51].reshape((6, 17, 3))

    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)


    out.write(frame)

print(f" Processing complete! Total frames written: {frame_count}")

cap.release()
out.release()


!ffmpeg -y -i movenet_output.avi -vcodec libx264 movenet_output.mp4

files.download("movenet_output.mp4")



TensorFlow version: 2.19.0
Loading MoveNet model...
Model loaded successfully!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Input video found!
Video properties: 1920x1080, 25 FPS
VideoWriter ready!
End of video reached
✅ Processing complete! Total frames written: 1464
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-l

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>